In [164]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.interpolate import interp1d
import random
from matplotlib import cm

In [165]:
## Frå githubben


def computeIntegral1D(a,b,func,type=0):
    """_summary_

    Args:
        a (float): start of integraion intervall
        b (float): end of integraion intervall
        func (function): function we want to integrate
        type (int, optional): what integral method we will use

    Returns:
        approx(float) : value of integral
    """
    h=b-a
    if type==0:
        approx = h/2 * (func(-h/(2*np.sqrt(3)) + (a+b)/2,a,b) + func(h/(2*np.sqrt(3)) +(a+b)/2,a,b))
    else: 
        approx = h/2 * (1/3*func(-h/2 + (a+b)/2,a,b) + 4/3*func((a+b)/2,a,b) + 1/3*func(h/2+(a+b)/2,a,b))
    return approx


### 2   Two dimensional problem

#### 2.1



In [166]:
class Phi:
    def __init__(self,index):
        self.index = index
    
    def func(self,x,a,b):
        h = b-a
        if self.index == 0:
            return (x-a)/h
        if self.index == 1:
            return (b-x)/h

class Dphi:
    def __init__(self,index):
        self.index = index
    
    def func(self,x,a,b):
        h = b-a
        if self.index == 0:
            return 1/h
        if self.index == 1:
            return -1/h
        
class Integrand2D:
    def __init__(self,index1,index2,type = "O"):
        self.index1 = index1
        self.index2 = index2
        self.type = type         # 0: phi, 1: dphi
    
    def func(self,x,a,b):
        if self.type == "O":
            return Phi(self.index1).func(x,a,b)*Phi(self.index2).func(x,a,b)
        if self.type == "D":
            return Dphi(self.index1).func(x,a,b)*Dphi(self.index2).func(x,a,b)

    

In [ ]:
def massMatrix(mesh, s):
    '''
    Computes the stiffness matrix

    
    '''
    n = len(mesh)
    M = sp.sparse.lil_matrix((n,n))
    for i in range(1,n):            # Sjekk om dette stemmer med oppgåva i 1D
        I1 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(0,0).func, s)
        I2 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(0,1).func, s)
        I3 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(1,1).func, s)

        M[i-1,i-1] += I1
        M[i-1,i] += I2
        M[i,i-1] += I2
        M[i,i] += I3
    
    return M.tocsr()

s = 1
n = 6

mesh = np.linspace(0,1,n)

print(massMatrix(mesh,s).toarray())


In [ ]:
def stiffnessMatrix(mesh, s):
    '''
    Computes the stiffness matrix

    
    '''
    n = len(mesh)
    A = sp.sparse.lil_matrix((n,n))
    for i in range(1,n):            # Sjekk om dette stemmer med oppgåva i 1D
        I1 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(0,0,"D").func, s)
        I2 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(0,1,"D").func, s)
        I3 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(1,1,"D").func, s)

        A[i-1,i-1] += I1
        A[i-1,i] += I2
        A[i,i-1] += I2
        A[i,i] += I3
    
    return A.tocsr()

s = 0
n = 6

mesh = np.linspace(0,1,n)

print(stiffnessMatrix(mesh,s).toarray())

In [169]:
def l(n,i,j):
    return (n+1)*j+i

In [170]:
def loadVector2D(mesh_x, mesh_y, s, f):
    """
    
    
    """
    n = len(mesh_x)-1
    F = np.zeros((n+1)**2)


    for j in range(0,n+1):
        for i in range(0,n+1):
            def G(y):

                def integr1(x,a,b):
                    return f(x,y)*Phi(0).func(x,a,b)
                def integr2(x,a,b):
                    return f(x,y)*Phi(1).func(x,a,b)
                


                if i == 0:
                    I1 = 0
                    I2 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr2,s)
                elif i == n:
                    I1 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr1,s)
                    I2 = 0
                else:
                    I1 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr1,s)
                    I2 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr2,s)
                return I1 + I2


            
            def integr3(y,a,b):
                return G(y)*Phi(0).func(y,a,b)
            def integr4(y,a,b):
                return G(y)*Phi(1).func(y,a,b)
            

            if j == 0:
                I3 = 0
                I4 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr4,s)
            elif j == n:
                I3 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr3,s)
                I4 = 0
            else:
                I3 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr3,s)
                I4 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr4,s)


            F[l(n,i,j)] = I3 + I4
    
    return F
        


In [ ]:
n = 3
s = 0

def f(x,y):
    return x*y 


meshx = np.linspace(0,1,n+1)
meshy = np.linspace(0,1,n+1)

F = loadVector2D(meshx,meshy,s,f)
print(F.reshape(n+1,n+1))




In [271]:
def applyNeumann2D(mesh_x,mesh_y,s,A,F,g):
    """
    Applies the Neumann conditions to F

    """
    F_new = F.copy()
    n = int(np.sqrt(len(F))) - 1

    for j in set((0,n)):
        for i in range(n+1):
            if j == 0:
                def integr0(x,a,b):
                    return g(x,0)*Phi(0).func(x,a,b)
                def integr1(x,a,b):
                    return g(x,0)*Phi(1).func(x,a,b)
                
                
                if i == 0:
                    I0 = 0
                    I1 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr1,s)
                elif i == n:
                    I0 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr0,s)
                    I1 = 0
                else:
                    I0 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr0,s)
                    I1 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr1,s)
                
                F_new[l(n,i,j)] += I0 + I1

            if j == n:
                def integr0(x,a,b):
                    return g(x,1)*Phi(0).func(x,a,b)
                def integr1(x,a,b):
                    return g(x,1)*Phi(1).func(x,a,b)
                
                if i == 0:
                    I0 = 0
                    I1 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr1,s)
                elif i == n:
                    I0 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr0,s)
                    I1 = 0
                else:
                    I0 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr0,s)
                    I1 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr1,s)
                
                F_new[l(n,i,j)] += I0 + I1

    for i in set((0,n)):
        for j in range(n+1):
            if i == 0:
                def integr0(y,a,b):
                    return g(0,y)*Phi(0).func(y,a,b)
                def integr1(y,a,b):
                    return g(0,y)*Phi(1).func(y,a,b)
                
                if j == 0:
                    I0 = 0
                    I1 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr1,s)
                elif j == n:
                    I0 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr0,s)
                    I1 = 0
                else:
                    I0 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr0,s)
                    I1 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr1,s)
                
                F_new[l(n,i,j)] += I0 + I1
            
            if i == n:
                def integr0(y,a,b):
                    return g(1,y)*Phi(0).func(y,a,b)
                def integr1(y,a,b):
                    return g(1,y)*Phi(1).func(y,a,b)
                
                if j == 0:
                    I0 = 0
                    I1 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr1,s)
                elif j == n:
                    I0 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr0,s)
                    I1 = 0
                else:
                    I0 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr0,s)
                    I1 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr1,s)
                
                F_new[l(n,i,j)] += I0 + I1




    return A, F_new
                



In [278]:
def FEMsolve2D(f,g,mesh_x,mesh_y,s=0):
    """
    Solves the finite element method for the 2D problem
    
    """

    M_x = massMatrix(mesh_x,s)
    M_y = massMatrix(mesh_y,s)
    A_x = stiffnessMatrix(mesh_x,s)
    A_y = stiffnessMatrix(mesh_y,s)

    A = sp.sparse.kron(A_y,M_x) + sp.sparse.kron(M_y, A_x)
    A[0,0] +=1           # Uniqueness: setting u_00 = 0
    A = A.tocsr()

    F = loadVector2D(mesh_x,mesh_y,s,f)

    A,F = applyNeumann2D(mesh_x,mesh_y,s,A,F,g)

    U = sp.sparse.linalg.spsolve(A,F)

    return U


In [281]:
n = 500
s = 0

def u(x,y):
    pi = np.pi
    return np.sin(2*pi*x)*np.sin(2*pi*y)
    # return np.cos(2*pi*x)*np.cos(2*pi*y)


def f(x,y):
    return 8*np.pi**2*u(x,y)

def g(x,y):
    pi = np.pi
    if x == 0:
        return -2*pi*np.sin(2*pi*y)
    if x == 1:
        return 2*pi*np.sin(2*pi*y)
    if y == 0:
        return -2*pi*np.sin(2*pi*x)
    if y == 1:
        return 2*pi*np.sin(2*pi*x)

    return 0


meshx = np.linspace(0,1,n+1)
meshy = np.linspace(0,1,n+1)

X,Y = np.meshgrid(meshx,meshy)

U = FEMsolve2D(f,g,meshx,meshy,s).reshape(n+1,n+1)



In [ ]:
ax= plt.figure().add_subplot(projection='3d')
plt.title("Numerical Solution for $u(x,y)$")
# ax.plot_surface(Y.T,X.T,u(X,Y).T, cmap = cm.coolwarm)
ax.plot_surface(Y.T,X.T,U.T, cmap = cm.coolwarm)

ax.set_ylabel("x")
ax.set_xlabel("y")
plt.show()